# 简介：基于W&B Sweeps的超参数调优
很难在高维超参数空间去寻找一组参数组合，令模型效果达到最优。W&B Sweeps通过自动化地遍历超参数组合(e.g. learning rate, batch size, number of hidden layers, optimizer type)，来寻找最佳参数组合令模型效果最优。
![超参数调优](https://i.imgur.com/WVKkMWw.png)
## 简介：Sweeps
通过W&B Sweeps实现超参数调优十分方便，主要有以下三步
- 定义sweep(e.g. search strategy)：
    - 在python文件中通过字典定义config
    - 通过yaml文件读入config
- 初始化sweep：初始化sweep，并且传入参数
- 运行sweep agent：将sweep_id传入wandb.agent(),然后进行超参数调优

# 第一步：登陆W&B

In [59]:
import wandb

wandb.login()

True

# 第二步：定义sweep参数

## 超参数优化方法
- grid search(网格搜索): 穷举法，在所有的参数组合中，通过循环便利，尝试每一种可能性，选取评价指标最好的参数输出
- random search(随即搜索)：在预先设定的定义域内随机选取超参数组合
- bayes search(贝叶斯搜索)：区别于grid search和bayes search相对独立的参数搜索方式，bayes search希望通过上一次的效果来决定下一次的参数选择，以提高搜索效率

In [60]:
# grid random bayes
sweep_config = {
    'method': 'random'
    }

如果选用bayes，我们需要定义metric作为评价指标，使它在参数搜索过程中变得更小或者更大
```
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
```

# 第三步：定义超参数搜索范围
在进行搜索之前，我们需要定义超参数的搜索范围

In [67]:
import math

parameters_dict = {
    'num_filter':{
        'values':[3, 4, 5]
        },
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
      }
    }

sweep_config['parameters'] = parameters_dict

有些参数，我们不需要搜索，但是也希望在sweep中定义，那我们可以直接赋值

In [68]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })

完成sweep的定义，打印一下

In [69]:
# 数据美化输出
from pprint import pprint

pprint(sweep_config)

{'method': 'random',
 'parameters': {'batch_size': {'distribution': 'q_log_uniform',
                               'max': 5.545177444479562,
                               'min': 3.4657359027997265,
                               'q': 1},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'num_filter': {'values': [3, 4, 5]},
                'optimizer': {'values': ['adam', 'sgd']}}}


# 第四步：初始化sweep

In [80]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")
wandb.init()

Create sweep with ID: natr6h12
Sweep URL: https://wandb.ai/8christmas8/pytorch-sweeps-demo/sweeps/natr6h12


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [27]:
import codecs
import pandas as pd
path_data_file = "/data/nlp-data/open_source_data/classification/toutiao_cat_data.txt"

text = []
label = []
with codecs.open(path_data_file, "r") as f:
    for line in f.readlines():
        line = line.strip()
        if not line or len(line) < 1:
            continue
        contents = line.split("_!_")
        label.append(contents[2])
        text.append("".join(contents[3:]))
dict_origin = {
    "text": text,
    "label": label
}
df_origin = pd.DataFrame(dict_origin,columns=["text", "label"])
df_origin

,text,label
0,"京城最值得你来场文化之旅的博物馆保利集团,马未都,中国科学技术馆,博物馆,新中国",news_culture
1,发酵床的垫料种类有哪些？哪种更好？,news_culture
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture
4,黄杨木是什么树？,news_culture
...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military
382685,"又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！以色列,普京,俄罗斯,叙利亚,车臣",news_world
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world


In [47]:
value_counts = df_origin["label"].value_counts().to_frame()
print(value_counts.shape)
value_counts

(15, 1)


,label
news_tech,41543
news_entertainment,39396
news_sports,37568
news_car,35785
news_game,29300
news_culture,28031
news_finance,27085
news_edu,27058
news_world,26909
news_military,24984


In [32]:
from tqdm import tqdm
from collections import Counter

words = "".join(list(df_origin["text"]))  
vocab = [w for w, f in Counter(words).most_common() if f > 1]
vocab = ['<pad>', '<unk>'] + vocab
index2word = {i: word for i, word in enumerate(vocab)}
word2index = {word: i for i, word in enumerate(vocab)}

In [33]:
max_length = 512
def sentence_2_ids(sentence:str):
    unk_id = word2index['<unk>']
    pad_id = word2index['<pad>']
    # 对句子进行数字化转换，对于未在词典中出现过的词用unk的index填充
    if len(sentence) > max_length:
        sentence = sentence[-max_length:]
    indexes = [word2index.get(word, unk_id) for word in sentence]
    if len(indexes) < max_length:
        indexes.extend([pad_id] * (max_length - len(indexes)))
    if len(indexes) > max_length:
        indexes = indexes[-max_length:]
    if len(indexes) != 512:
        print(indexes)
    return indexes

In [41]:
import torch
class DatasetTextCNN(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return self.encodings[idx], self.labels[idx]
    def __len__(self):
        return len(self.encodings)

In [37]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(label)

LabelEncoder()

In [42]:
from pandas import DataFrame
def df_2_dataset_textcnn(df: DataFrame):
    x = list(df["text"])
    df["label_id"] = le.transform(df["label"].tolist())
    y = list(df["label_id"])
    text = []
    for ele in x:
        words = sentence_2_ids(ele)
        text.append(words)
    result = DatasetTextCNN(text, y)
    return result

In [74]:
from sklearn.model_selection import train_test_split


df_train, df_test = train_test_split(df_origin, test_size=0.1, stratify=df_origin[['label']])
df_train, df_eval = train_test_split(df_train, test_size=0.1, stratify=df_train[['label']])

# 为了减少训练时间，这里训练集只取20%
df_train, _ = train_test_split(df_train, test_size=0.8, stratify=df_train[['label']])


pprint(df_train.shape)
pprint(df_test.shape)
pprint(df_eval.shape)

(61995, 2)
(38269, 2)
(34442, 2)


In [43]:
dataset_train = df_2_dataset_textcnn(df_train)
dataset_eval = df_2_dataset_textcnn(df_eval)
dataset_test = df_2_dataset_textcnn(df_test)

In [44]:
from torch.utils.data import DataLoader
def collate_fn(batch):
    """
    对batch数据进行处理
    :param batch: [一个getitem的结果，getitem的结果,getitem的结果]
    :return: 元组
    """
    text, label = zip(*batch)
    new_text = torch.LongTensor(text)
    new_label = torch.LongTensor(label)
    return new_text, new_label

In [49]:
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F

use_cuda = True if torch.cuda.is_available() else False
MAX_LENGTH = 512
FILTERS = [2,3,4,5]
NUM_LABEL = value_counts.shape[0]

class TextCNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, dropout, num_filter):
        super(TextCNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #卷积层
        if use_cuda and torch.cuda.is_available():
            self.convs = [
                nn.Sequential(
                    nn.Conv1d(embedding_dim, num_filter, filter_size),
                    nn.ReLU(),
                    nn.MaxPool1d(kernel_size=MAX_LENGTH - filter_size + 1)
                ).cuda()
                for filter_size in FILTERS]
        else:
            self.convs = [
                nn.Sequential(
                    nn.Conv1d(embedding_dim, num_filter, filter_size),
                    nn.ReLU(),
                    nn.MaxPool1d(kernel_size=MAX_LENGTH - filter_size + 1)
                )
                for filter_size in FILTERS]

        # 正则化处理
        self.dropout = nn.Dropout(dropout, inplace=True)

        # 分类层
        self.fc = nn.Linear(num_filter*len(FILTERS), NUM_LABEL)

    def conv_and_pool(self,x,conv):
        x = F.relu(conv(x))
        # 池化层
        x = F.max_pool1d(x,x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2).contiguous()
        x = torch.cat([conv_relu_pool(x) for conv_relu_pool in self.convs], dim=1).squeeze(2)
        # x = torch.cat([self.conv_and_pool(x, conv) for conv in self.convs], 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [76]:
import os
import torch
import wandb
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import List
from sklearn import metrics
import torch.nn.functional as F

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "text-cnn"

class Trainer(object):
    def __init__(self,
                model,
                criterion,
                optimizer,
                epochs,
                learning_rate,
                dataset_train,
                dataset_eval,
                dataset_test,
                labels: List,
                dir_output,
                dir_model,
                is_resume: bool=False) -> None:
        super(Trainer, self).__init__()

        # FIXME read parameter from bash

        self.model = model
        self.learning_rate = learning_rate
        self.criterion = criterion
        self.optimizer = optimizer
        self.dataset_train = dataset_train
        self.dataset_eval = dataset_eval
        self.dataset_test = dataset_test
        self.epochs = epochs
        self.iteration = 0
        self.labels = labels
        self.dir_model = dir_model
        self.dir_output = dir_output
        self.best_score = 0
        self.is_resume = is_resume
    
    def run(self):
        # 训练
        self.train()
        # 验证
        # self.test()
    
    def train(self):
        print("*********************************************************")
        print("Train")
        self.model.train()
        start_epoch = 0
        # 继续训练
        if self.is_resume:
            path_checkpoint = os.path.join(self.dir_model, "best", MODEL_NAME)  # 断点路径
            checkpoint = torch.load(path_checkpoint)  # 加载断点
            self.model.load_state_dict(checkpoint['net'])  # 加载模型可学习参数
            self.learning_rate.load_state_dict(checkpoint['lr_schedule'])
            self.optimizer.load_state_dict(checkpoint['optimizer'])  # 加载优化器参数
            start_epoch = checkpoint['epoch']  # 设置开始的epoch
        
        for epoch in tqdm(range(start_epoch, self.epochs)):
            for batch_x, batch_y in self.dataset_train:
                batch_x, batch_y = batch_x.to(DEVICE), batch_y.to(DEVICE)
                pred = self.model(batch_x)
                loss = self.criterion(pred, batch_y)
                wandb.log({"batch loss": loss.item()})
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            # Eval model score
            acc, eval_loss = self.__eval()
            wandb.log({"loss": eval_loss, "epoch": epoch})
            # Update Model
            # if acc > self.best_score:
            #     self.best_score = acc
            #     self.save_model(epoch, True)
                
    def compute_loss(self, input, label):
        outputs = self.model(input.to(DEVICE)).to(DEVICE)
        loss = F.cross_entropy(outputs, label.to(DEVICE))
        return loss

    def __eval(self):
        print("*********************************************************")
        print("Eval")
        self.model.eval()
        loss_total = 0
        predict_all = np.array([], dtype=int)
        label_all = np.array([], dtype=int)
        with torch.no_grad():
            for text, label in self.dataset_eval:
                outputs = self.model(text.to(DEVICE)).to(DEVICE)
                loss = self.compute_loss(text, label)
                loss_total += loss
                label = label.data.cpu().numpy()
                predict = torch.max(outputs.data, 1)[1].cpu().numpy()
                label_all = np.append(label_all, label)
                predict_all = np.append(predict_all, predict)
        
        ave = "macro" if len(set(predict_all)) > 2 else "binary"
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(label_all, predict_all, average=ave)
        loss = loss_total / len(self.dataset_eval)
        pprint('evaluation loss : {:.6f}, precision : {:.6f}, recall : {:.6f}, fscore : {:.6f},'.format(loss, precision, recall, fscore))
        return precision, loss

    # def save_model(self, epoch:int, is_best: bool):
    #     state = {
    #             'epoch': epoch,
    #             'net': self.model.state_dict(),
    #             'optimizer': self.optimizer.state_dict(),
    #             'lr_schedule': self.learning_rate.state_dict()
    #         }
    #     if is_best:
    #         torch.save(state, os.path.join(self.dir_model, "best", MODEL_NAME))
    #     else:
    #         torch.save(state, os.path.join(self.dir_model, "ckpt_"+str(epoch), MODEL_NAME))


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kqid32m1/file_stream
NoneType: None


In [51]:
import torch.optim as optim

def build_optimizer(model, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(model.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(model.parameters(),
                               lr=learning_rate)
    return optimizer

In [57]:
VOCAB_SIZE = 10000

EMBEDDING_DIM = 200

MAX_LENGTH = 512

model = None
labels = le.inverse_transform([x for x in range(value_counts.shape[0])])
def train(config=None):
    with wandb.init(config=None):
        config = wandb.config
        print(config)
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        model = TextCNN(VOCAB_SIZE, EMBEDDING_DIM, config.dropout, config.num_filter).to(DEVICE)
        dataloader_train = DataLoader(dataset_train, batch_size=config.batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)
        dataloader_eval = DataLoader(dataset_eval, batch_size=config.batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)
        dataloader_test = DataLoader(dataset_test, batch_size=config.batch_size, shuffle=True, num_workers=0, collate_fn=collate_fn)
        trainer = Trainer(
            model = model,
            criterion = nn.CrossEntropyLoss().to(DEVICE),
            optimizer = build_optimizer(model, config.optimizer, config.learning_rate),
            epochs = 4,
            learning_rate = 2e-4,
            dataset_train = dataloader_train,
            dataset_eval = dataloader_eval,
            dataset_test = dataloader_test,
            labels = labels,
            dir_output = "",
            dir_model = ""
        )
        trainer.run()

In [81]:
wandb.agent(sweep_id, train, count=50)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: dcg1g8r5 with config:
wandb: 	batch_size: 80
wandb: 	dropout: 0.3
wandb: 	epochs: 1
wandb: 	learning_rate: 0.06059954661540782
wandb: 	num_filter: 3
wandb: 	optimizer: sgd
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'batch_size': 80, 'dropout': 0.3, 'epochs': 1, 'learning_rate': 0.06059954661540782, 'num_filter': 3, 'optimizer': 'sgd'}
*********************************************************
Train


  0%|          | 0/4 [00:00<?, ?it/s]

*********************************************************
Eval


/mnt/nfsshare/miniconda3/envs/wf_pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 25%|██▌       | 1/4 [02:10<06:32, 130.88s/it]

...

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:r

*********************************************************
Eval


/mnt/nfsshare/miniconda3/envs/wf_pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 4/4 [08:20<00:00, 125.00s/it]


('evaluation loss : 0.598149, precision : 0.770067, recall : 0.756610, fscore '
 ': 0.761437,')


wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/8christmas8/pytorch-sweeps-demo/kpzhd8l3/file_stream
NoneType: None
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:r